## Load the actual MATRES data created/annotated by Qiang Ning

In [29]:
MATRES_timebank = '/shared/why16gzl/logic_driven/EMNLP_2020/MATRES/timebank.txt'
MATRES_aquaint = '/shared/why16gzl/logic_driven/EMNLP_2020/MATRES/aquaint.txt'
MATRES_platinum = '/shared/why16gzl/logic_driven/EMNLP_2020/MATRES/platinum.txt'
eiid_to_event_trigger_train = {}
eiid_pair_to_label_train = {}
eiid_to_event_trigger_test = {}
eiid_pair_to_label_test = {}
MATRES_data_cases_train = []
MATRES_data_cases_test = []
temp_label_map = {"BEFORE": 0, "AFTER": 1, "EQUAL": 2, "VAGUE": 3}

def MATRES_READER(file, eiid_to_event_trigger, eiid_pair_to_label):
    with open(file, "r") as f_matres:
        content = f_matres.read().split("\n")
        for rel in content:
            rel = rel.split("\t")
            fname = rel[0]
            trigger1 = rel[1]
            trigger2 = rel[2]
            eiid1 = int(rel[3])
            eiid2 = int(rel[4])
            tempRel = temp_label_map[rel[5]]

            if fname not in eiid_to_event_trigger:
                eiid_to_event_trigger[fname] = {}
                eiid_pair_to_label[fname] = {}
            eiid_pair_to_label[fname][(eiid1, eiid2)] = tempRel
            if eiid1 not in eiid_to_event_trigger[fname].keys():
                eiid_to_event_trigger[fname][eiid1] = trigger1
            if eiid2 not in eiid_to_event_trigger[fname].keys():
                eiid_to_event_trigger[fname][eiid2] = trigger2

MATRES_READER(MATRES_timebank, eiid_to_event_trigger_train, eiid_pair_to_label_train)  
#MATRES_READER(MATRES_aquaint, eiid_to_event_trigger_train, eiid_pair_to_label_train)  
MATRES_READER(MATRES_platinum, eiid_to_event_trigger_test, eiid_pair_to_label_test)  

In [176]:
if "wsj_0695" in eiid_to_event_trigger_train.keys():
    print("yes")

yes


In [197]:
if '2041' in eiid_to_event_trigger_train["APW19980322.0749"]:
    print("yes")

yes


## verify whether token's first char matches the starting char position in the article

In [2]:
def verify(token_list, My_Text, debugging):
    count_error = 0
    for token in token_list:
        if debugging: print(token)
        if token[1] == ' ':
            continue
        if token[1][0] != ' ':
            if My_Text[token[2]] == token[1][0]: # token[2] is the starting char position in My_Text; token[1] is the word
                count_error = count_error
            else:
                count_error += 1
                print(token)
        else:
            if My_Text[token[2]] == token[1][1]:
                count_error = count_error
            else:
                count_error += 1
                print(token)
    #print("count_error:",count_error)
    return count_error

In [8]:
from os import listdir
from os.path import isfile, join

bert_emb_path = '/shared/why16gzl/logic_driven/EMNLP-2020/MATRES/bertEMB/'
dict_path = '/shared/why16gzl/logic_driven/EMNLP-2020/MATRES/event_mention/'
token_dict_path = '/shared/why16gzl/logic_driven/EMNLP-2020/MATRES/token_dict_path/'
import transformers
from transformers import RobertaTokenizer, RobertaModel
from torch.nn.utils.rnn import pad_sequence
import torch
import os
import pickle
import nltk
from nltk.tokenize import sent_tokenize
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', unk_token='<unk>')
model = RobertaModel.from_pretrained('roberta-base')

## Get RoBERTa embeddings for text in MATRES, the data is actually from TempEval3 https://www.cs.york.ac.uk/semeval-2013/task1/index.php%3Fid=data.html

### We did not find the annotated file named "nyt_20130321_sarcozy.tml", but only find the original text. Hence we temporarily eliminate this file from MATRES 
/shared/why16gzl/logic_driven/EMNLP-2020/MATRES/platinum.txt

In [9]:
mypath_TB = '/shared/why16gzl/logic_driven/EMNLP-2020/MATRES/TBAQ-cleaned/TimeBank/' # after correction
onlyfiles_TB = [f for f in listdir(mypath_TB) if isfile(join(mypath_TB, f))]
mypath_AQ = '/shared/why16gzl/logic_driven/EMNLP-2020/MATRES/TBAQ-cleaned/AQUAINT/' 
onlyfiles_AQ = [f for f in listdir(mypath_AQ) if isfile(join(mypath_AQ, f))]
mypath_PL = '/shared/why16gzl/logic_driven/EMNLP-2020/MATRES/te3-platinum/'
onlyfiles_PL = [f for f in listdir(mypath_PL) if isfile(join(mypath_PL, f))]

onlyfiles = onlyfiles_PL
#onlyfiles = ["WSJ_20130322_159.tml"]
mypath = mypath_PL
from numpy import array
from numpy import argmax
import numpy
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
"""
file = "./pos-tags.txt"
pos_tags_list = []
with open(file, 'r') as f:
    content = f.readlines()
    for line in content:
        pos_tags_list.append(line.split(" ")[4])
        
tag_to_num = {tag:i for i, tag in enumerate(sorted(pos_tags_list))} 
num_to_tag = {i:tag for i, tag in enumerate(sorted(pos_tags_list))}        
        
# define example
values = array(pos_tags_list)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

zero_35 = [0.] * 35
zero_35 = numpy.asarray(zero_35)
def POSTAG_TO_ONEHOT(pos_tag, onehot_encoded, tag_to_num):
    if pos_tag in tag_to_num.keys():
        num = tag_to_num[pos_tag]
        return onehot_encoded[num]
    else:
        return zero_35



"""
import pickle
POSTAG_ONEHOT_dict = pickle.load(open("/shared/why16gzl/logic_driven/EMNLP-2020/MATRES/TBAQ-cleaned/POSTAG_ONEHOT.dict", "rb"))
zero_18 = [0.] * 18
zero_18 = numpy.asarray(zero_18)
def POSTAG_TO_ONEHOT(pos_tag):
    if pos_tag in POSTAG_ONEHOT_dict.keys():
        return POSTAG_ONEHOT_dict[pos_tag]
    else:
        return zero_18


import xml.etree.ElementTree as ET

count = 0
debugging = 0
longest_sent = 0
for fname in onlyfiles:
    print(fname)
    count += 1
    #print(count)
    tree = ET.parse(mypath+fname)
    root = tree.getroot()
    MY_STRING = str(ET.tostring(root))
    
    # eID is the id for event in each article; eiid is the event id in the timebank/aquaint data
    # <MAKEINSTANCE eventID="e1" eiid="ei415" tense="PRESENT" aspect="PERFECTIVE" polarity="POS" pos="VERB" />
    eID_to_eiid = {}
    article_event = []
    for makeinstance in root.findall('MAKEINSTANCE'):
        instance_str = str(ET.tostring(makeinstance)).split(" ")
        try:
            assert instance_str[3].split("=")[0] == "eventID"
            assert instance_str[2].split("=")[0] == "eiid"
            eiid = int(instance_str[2].split("=")[1].replace("\"", "")[2:])
            eID = instance_str[3].split("=")[1].replace("\"", "")
        except:
            for i in instance_str:
                if i.split("=")[0] == "eventID":
                    eID = i.split("=")[1].replace("\"", "")
                if i.split("=")[0] == "eiid":
                    eiid = int(i.split("=")[1].replace("\"", "")[2:])
        eID_to_eiid[eID] = eiid
    #print("len(eID_to_eiid)",len(eID_to_eiid))    
    start = MY_STRING.find("<TEXT>") + 6
    end = MY_STRING.find("</TEXT>")
    MY_TEXT = MY_STRING[start:end]
    while MY_TEXT[0] == " ":
        MY_TEXT = MY_TEXT[1:]
    MY_TEXT = MY_TEXT.replace("\\n", " ")
    MY_TEXT = MY_TEXT.replace("\\'", "\'")
    MY_TEXT = MY_TEXT.replace("  ", " ")
    MY_TEXT = MY_TEXT.replace(" ...", "...")
    
    # eID is the id for event in each article; charID is the starting char position for each event
    # <EVENT eid="e1" class="I_ACTION">predicted</EVENT>
    eID_to_charID = {}
    while MY_TEXT.find("<") != -1:
        start = MY_TEXT.find("<")
        end = MY_TEXT.find(">")
        if MY_TEXT[start + 1] == "E":
            event_description = MY_TEXT[start:end].split(" ")
            eid = (event_description[2].split("="))[1].replace("\"", "")
            eID_to_charID[eid] = start
            MY_TEXT = MY_TEXT[:start] + MY_TEXT[(end + 1):]
        else:
            MY_TEXT = MY_TEXT[:start] + MY_TEXT[(end + 1):]
    #print("eID_to_charID: ", eID_to_charID)    
    if debugging: print(MY_TEXT)
    # k = 695 # the 695-th character is: s
    # said: eiid = 431, eID = e26 appears in ABC19980120.1830.0957
    #print(MY_TEXT[k-5:k+5]) # " has said "
    #print(eiid_to_event_trigger['430']) eiid 430 appears in APW19980227.0487
    # so we need to construct a dictionary for each article
    
    # verify whether token's first char matches the starting char position in the article
    # eID_to_charID is within the context of each article 
    if len(onlyfiles) > 30:
        eiid_to_event_trigger = eiid_to_event_trigger_train
    else:
        eiid_to_event_trigger = eiid_to_event_trigger_test
    count_error = 0
    #print(eID_to_eiid)
    ############################################################################
    #eiid_to_event_trigger = eiid_to_event_trigger_train
    for eID, charID in eID_to_charID.items():
        file_name = fname.replace(".tml", "")
        #print(eiid_to_event_trigger[file_name])
        if eID in eID_to_eiid.keys():
            #print(eID)
            if file_name in eiid_to_event_trigger.keys():
                #print(file_name)
                if eID_to_eiid[eID] in eiid_to_event_trigger[file_name].keys():
                    #print("eID_to_eiid: ", eID_to_eiid[eID])
                    trigger = eiid_to_event_trigger[file_name][eID_to_eiid[eID]]
                    # eID, eiid, charID, trigger word
                    article_event.append((eID, eID_to_eiid[eID], charID, trigger))
                    if MY_TEXT[charID] == trigger[0]:
                        count_error = count_error
                    else:
                        count_error += 1
    if count_error == 0:
        count_error = 0
    else:
        print("failed!!!!!!!!!!!!!!")
    if debugging: print(article_event)
    sent_tokenized_text = sent_tokenize(MY_TEXT)
    token_index = 0
    sent_index = 0
    token_list = []
    token_dict = {}
    sent_emb_list = []
    span_counter = 0
    for sent in sent_tokenized_text:
        #if debugging: print(sent)
        # '\n' and ' ' are both one char, so it doesn't influence the span starting position 
        #sent_to_be_encoded = sent.replace('\n', '')
        if debugging: print(sent)
            
        ### POS Tag One-Hot 
        tok=nltk.tokenize.word_tokenize(sent) 
        pos=nltk.pos_tag(tok)
        sent_pos_tag_dict = {}
        sent_word_dict = {}
        index_here = 0
        for (word, POS) in pos:
            ord_first_char = ord(word[0])
            POS_ONEHOT = POSTAG_TO_ONEHOT(POS)
            if POS_ONEHOT.sum() > 0 and ((ord_first_char >= 65 and ord_first_char <= 90) or (ord_first_char >= 97 and ord_first_char <= 122) or (ord_first_char >= 48 and ord_first_char <= 57)):
                sent_pos_tag_dict[index_here] = POSTAG_TO_ONEHOT(POS)
                sent_word_dict[index_here] = word
                index_here += 1

        encoded = tokenizer.encode(sent) 
        if len(encoded) > longest_sent:
            longest_sent = len(encoded)
        input_ids = torch.tensor(encoded).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids)
        last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
        last_hidden_states = last_hidden_states.view(-1, 768)
        #sent_emb_list.append(last_hidden_states.view(-1, 768))
        #print(last_hidden_states.view(-1, 768).size())
        token_num = list(last_hidden_states.size())[0]
        POS_TAG_ONEHOT = torch.zeros(token_num, 18)

        word_index = 0
        current_word_index = 0

        for index, i in enumerate(encoded):
            list_ = [i]
            token = tokenizer.decode(list_)
            if token == "<s>" or token == "</s>":
                #if debugging: print(token)
                continue
            if token[0] == " ": 
                token_word = token[1:]
                if word_index in sent_word_dict:
                    #print(sent_word_dict[word_index])
                    #print(sent_word_dict)
                    if sent_word_dict[word_index][0] == token[1]:
                        POS_TAG_ONEHOT[index] = torch.tensor(sent_pos_tag_dict[word_index])
                        current_word_index = word_index
                        word_index += 1
                span_start = span_counter + 1
            else:
                POS_TAG_ONEHOT[index] = torch.tensor(sent_pos_tag_dict[current_word_index])
                span_start = span_counter
                token_word = token
            span_counter += len(token)
            span_end = span_counter - 1
            if debugging: print(token_index, token, span_start, span_end)
            token_list.append((token_index, token, span_start, span_end, 0))
            token_dict[span_start] = [index, token, span_end, 0, sent_index]
            if debugging: print("token:",token)
            if debugging: print("span_start:",span_start)
            assert token_word[0] == MY_TEXT[span_start]
            token_index += 1

        sent_emb_list.append(torch.cat((last_hidden_states, POS_TAG_ONEHOT), dim=1))     
        sent_index += 1
        
    #print("number of tokens:", len(token_list))
    event_mention_dict = {}
    #print("len(article_event):",len(article_event))
    for art_eve in article_event: # article_event.append((eID, eID_to_eiid[eID], charID, trigger))
        if debugging: 
            print(int(art_eve[2]))
        token_dict[int(art_eve[2])][3] = 1
        eiid = int(art_eve[1])
        eID = art_eve[0]
        #print("eiid:",eiid)
        #<EventMentionInfo> <ID>25</ID> <AnchorText>took</AnchorText> <Type>Occurrence</Type> <Position>1154</Position </EventMentionInfo>
        # event_id, BertTokenized_token_id, sent_id, event_mention
        # key: eID; [token_index, sent_index, token, eiid, trigger]
        event_mention_dict[eiid] = [token_dict[int(art_eve[2])][0], token_dict[int(art_eve[2])][4], token_dict[int(art_eve[2])][1], eID_to_eiid[eID], eiid_to_event_trigger[fname.replace(".tml", "")][eID_to_eiid[eID]]]
    #print(token_dict)
    if not verify(token_list, MY_TEXT, debugging):
        print("verify success")
    else:
        print("verify fail")
             
    #print(event_mention_dict)
    with open(bert_emb_path + fname.replace('.tml', '.pickle'), 'wb') as file_out:
        c = torch.ones(120, 768+18)
        sent_emb_list.append(c)
        art_emb = pad_sequence(sent_emb_list, batch_first=True)
        pickle.dump(art_emb, file_out)
        
    with open(dict_path + fname.replace('.tml', '.event'), 'wb') as f_out:
        pickle.dump(event_mention_dict, f_out)
    
    with open(token_dict_path + fname.replace('.tml', '.tokenDICT'), 'wb') as f_out:
        pickle.dump(token_dict, f_out)
        
print("longest_sent:", longest_sent)        

bbc_20130322_1150.tml
verify success
CNN_20130322_314.tml
verify success
bbc_20130322_721.tml
verify success
nyt_20130322_strange_computer.tml
verify success
bbc_20130322_332.tml
verify success
nyt_20130321_china_pollution.tml
verify success
nyt_20130321_cyprus.tml
verify success
WSJ_20130321_1145.tml
verify success
CNN_20130321_821.tml
verify success
WSJ_20130318_731.tml
verify success
nyt_20130321_sarkozy.tml
verify success
CNN_20130322_1003.tml
verify success
bbc_20130322_1600.tml
verify success
AP_20130322.tml
verify success
bbc_20130322_1353.tml
verify success
CNN_20130322_1243.tml
verify success
CNN_20130322_248.tml
verify success
WSJ_20130322_804.tml
verify success
nyt_20130321_women_senate.tml
verify success
WSJ_20130322_159.tml
verify success
longest_sent: 81


## The function of the cell below is the same as the first half of:
/home1/w/why16gzl/KAIROS/event_abstraction/baselines/train_test_gen.py

In [33]:
mypath_TB = '/shared/why16gzl/logic_driven/EMNLP_2020/MATRES/TBAQ-cleaned/TimeBank/' # after correction
onlyfiles_TB = [f for f in listdir(mypath_TB) if isfile(join(mypath_TB, f))]
MATRES_data_cases_train = []
count_file = 0
for fname in eiid_to_event_trigger_train.keys():
    print("length of train_set_MATRES:", len(MATRES_data_cases_train))
    print(fname)
    count_file += 1
    #with open(dict_path+fname+".event", "rb") as f:
    #   event_mention_dict = pickle.load(f)
    #    print(event_mention_dict)
    eiid_to_event_trigger_dict = eiid_to_event_trigger_train[fname]
    #x: eiid; 
    for x in eiid_to_event_trigger_dict.keys():
        for y in eiid_to_event_trigger_dict.keys():
            for z in eiid_to_event_trigger_dict.keys():
                if x!=y and y!=z and x!=z:
                    if (x, y) in eiid_pair_to_label_train[fname].keys() and (y, z) in eiid_pair_to_label_train[fname].keys() and (x, z) in eiid_pair_to_label_train[fname].keys():
                        assert type(x) == type(1)
                        xy = eiid_pair_to_label_train[fname][(x, y)]
                        yz = eiid_pair_to_label_train[fname][(y, z)]
                        xz = eiid_pair_to_label_train[fname][(x, z)]
                        #MATRES_data_cases_train.append((count_file, x, y, z, event_mention_dict[x][0], event_mention_dict[y][0], event_mention_dict[z][0], xy, yz, xz, -1, -1, fname))
                        if fname+".tml" in onlyfiles_TB: 
                            MATRES_data_cases_train.append((x, y, z))
                            print(x, y, z)
                    
                        '''
for fname in eiid_to_event_trigger_test.keys():
    print(fname)
    count_file += 1
    with open(dict_path+fname+".event", "rb") as f:
        event_mention_dict = pickle.load(f)
    eiid_to_event_trigger_dict = eiid_to_event_trigger_test[fname]
    #print("eiid_to_event_trigger_dict", eiid_to_event_trigger_dict)
    #print("event_mention_dict:", event_mention_dict)
    for x in eiid_to_event_trigger_dict.keys():
        for y in eiid_to_event_trigger_dict.keys():
            if x!=y:
                if (x, y) in eiid_pair_to_label_test[fname].keys():
                    xy = eiid_pair_to_label_test[fname][(x, y)]
                    if fname == "WSJ_20130322_159":
                        assert type(x) == type(1)
                        #print(eiid_to_event_trigger_test[fname])
                    MATRES_data_cases_test.append((count_file, x, y, x, event_mention_dict[x][0], event_mention_dict[y][0], -1, xy, xy, xy, -1, -1, fname))
                    
print(len(MATRES_data_cases_train), 'event triples out of', len(eiid_to_event_trigger_train.keys()), 'articles.')
print(len(MATRES_data_cases_test), 'event pairs out of', len(eiid_to_event_trigger_test.keys()), 'articles.')
with open("/shared/why16gzl/logic_driven/EMNLP-2020/MATRES_data_cases_train.pickle", 'wb') as fOUT:
    pickle.dump(MATRES_data_cases_train, fOUT)
with open("/shared/why16gzl/logic_driven/EMNLP-2020/MATRES_data_cases_test.pickle", 'wb') as fOUT:
    pickle.dump(MATRES_data_cases_test, fOUT)
    '''

length of train_set_MATRES: 0
ABC19980120.1830.0957
415 417 418
415 417 419
415 418 419
417 418 419
419 420 421
420 421 422
420 421 423
420 422 423
421 422 423
425 426 429
426 429 431
435 437 441
437 441 442
437 441 444
437 442 444
441 442 444
length of train_set_MATRES: 16
wsj_0791
187 190 191
190 191 192
190 191 194
190 191 196
190 192 194
190 192 196
190 194 196
191 192 194
191 192 196
191 194 196
192 194 196
199 200 201
200 201 202
length of train_set_MATRES: 29
VOA19980305.1800.2603
433 436 437
438 441 445
456 458 460
460 462 463
462 463 464
462 463 465
462 464 465
463 464 465
464 465 467
482 484 485
482 484 487
482 484 488
482 485 487
482 485 488
482 487 488
484 485 487
484 485 488
484 487 488
485 487 488
length of train_set_MATRES: 48
VOA19980303.1600.0917
2222 2223 2226
2226 2230 2231
2226 2230 2232
2226 2230 2235
2226 2231 2232
2226 2231 2235
2226 2232 2235
2230 2231 2232
2230 2231 2235
2230 2231 2238
2230 2232 2235
2230 2232 2238
2230 2235 2238
2231 2232 2235
2231 2232 2238
2

2466 2467 2469
2467 2469 2471
length of train_set_MATRES: 543
wsj_0332
154 157 159
161 163 164
163 164 166
length of train_set_MATRES: 546
NYT19980206.0466
516 525 527
550 552 553
length of train_set_MATRES: 548
wsj_0542
361 362 364
361 362 365
361 362 369
361 364 365
361 364 369
361 365 369
362 364 365
362 364 369
362 365 369
364 365 369
364 365 370
364 365 371
364 369 370
364 369 371
364 370 371
365 369 370
365 369 371
365 370 371
369 370 371
370 371 376
370 371 378
370 371 380
370 376 378
370 376 380
370 378 380
371 376 378
371 376 380
371 378 380
376 378 380
376 378 381
376 378 383
376 378 384
376 380 381
376 380 383
376 380 384
376 381 383
376 381 384
376 383 384
378 380 381
378 380 383
378 380 384
378 381 383
378 381 384
378 383 384
380 381 383
380 381 384
380 383 384
381 383 384
381 383 386
381 383 387
381 384 386
381 384 387
381 386 387
383 384 386
383 384 387
383 386 387
384 386 387
386 387 389
386 387 390
386 389 390
387 389 390
389 390 392
length of train_set_MATRES: 610
wsj

208 211 213
208 212 213
211 212 213
length of train_set_MATRES: 1383
wsj_0751
138 140 141
138 140 143
138 141 143
140 141 143
141 143 144
141 143 146
141 144 146
143 144 146
length of train_set_MATRES: 1391
AP900816-0139
1566 1567 1569
1567 1569 1570
1567 1569 1571
1567 1569 1572
1567 1570 1571
1567 1570 1572
1567 1571 1572
1523 1524 1526
1523 1524 1531
1523 1524 1533
1523 1524 1534
1523 1526 1531
1523 1526 1533
1523 1526 1534
1523 1531 1533
1523 1531 1534
1523 1533 1534
1524 1526 1531
1524 1526 1533
1524 1526 1534
1524 1531 1533
1524 1531 1534
1524 1533 1534
1526 1531 1533
1526 1531 1534
1526 1533 1534
1531 1533 1534
1531 1533 1543
1531 1534 1543
1533 1534 1543
1543 1547 1548
1543 1547 1551
1543 1548 1551
1547 1548 1551
1547 1548 1553
1547 1551 1553
1548 1551 1553
1553 1555 1556
1555 1556 1566
1555 1556 1561
1555 1561 1566
1556 1561 1566
1561 1566 1567
1561 1566 1569
1561 1567 1569
1569 1570 1571
1569 1570 1572
1569 1571 1572
1570 1571 1572
1570 1571 1573
1570 1571 1574
1570 1572 1573

2038 2039 2042
2042 2045 2047
2045 2047 2051
2051 2055 2056
2055 2056 2059
2059 2062 2063
2059 2062 2064
2059 2063 2064
2062 2063 2064
2066 2067 2071
2072 2074 2076
2072 2074 2077
2072 2074 2083
2072 2076 2077
2072 2076 2083
2072 2077 2083
2074 2076 2077
2074 2076 2083
2074 2077 2083
2076 2077 2083
2086 2087 2088
2087 2088 2089
2087 2088 2090
2087 2089 2090
2088 2089 2090
2089 2090 2093
2089 2090 2094
2089 2093 2094
2090 2093 2094
2098 2102 2104
2102 2104 2106
2106 2107 2108
2107 2108 2110
length of train_set_MATRES: 2168
wsj_1042
81 82 83
83 84 85
84 85 87
length of train_set_MATRES: 2171
wsj_0263
2115 2117 2118
2115 2117 2120
2115 2118 2120
2117 2118 2120
2117 2118 2121
2117 2120 2121
2118 2120 2121
length of train_set_MATRES: 2178
wsj_0752
95 100 103
length of train_set_MATRES: 2179
wsj_0292
321 324 325
324 325 328
328 331 332
331 332 338
338 339 340
338 339 341
338 340 341
339 340 341
339 340 343
339 340 345
339 341 343
339 341 345
339 343 345
340 341 343
340 341 345
340 343 345
34

2468 2470 2473
2468 2472 2473
2470 2472 2473
2472 2473 2475
2478 2479 2480
2478 2479 2481
2478 2480 2481
2479 2480 2481
2479 2480 2486
2479 2481 2486
2480 2481 2486
2486 2488 2493
2488 2493 2495
2502 2503 2504
2502 2503 2506
2502 2503 2509
2502 2503 2512
2502 2504 2506
2502 2504 2509
2502 2504 2512
2502 2506 2509
2502 2506 2512
2502 2509 2512
2503 2504 2506
2503 2504 2509
2503 2504 2512
2503 2506 2509
2503 2506 2512
2503 2509 2512
2504 2506 2509
2504 2506 2512
2504 2509 2512
2506 2509 2512
2509 2512 2513
length of train_set_MATRES: 3358
PRI19980306.2000.1675
196 198 200
198 200 203
203 206 211
203 206 212
203 211 212
206 211 212
206 211 213
206 211 215
206 212 213
206 212 215
206 213 215
211 212 213
211 212 215
211 213 215
212 213 215
length of train_set_MATRES: 3373
wsj_0924
161 164 165
length of train_set_MATRES: 3374
wsj_0027
1989 1990 1993
1989 1990 2001
1989 1993 2001
1990 1993 2001
2002 2004 2005
2004 2005 2007
2007 2010 2011
2010 2011 2013
2021 2023 2024
2021 2023 2026
2021 2024

343 344 351
351 352 354
352 354 356
352 354 357
352 356 357
354 356 357
356 357 361
356 357 363
356 357 364
356 357 365
356 357 366
356 357 368
356 357 369
356 357 370
356 361 363
356 361 364
356 361 365
356 361 366
356 361 368
356 361 369
356 361 370
356 363 364
356 363 365
356 363 366
356 363 368
356 363 369
356 363 370
356 364 365
356 364 366
356 364 368
356 364 369
356 364 370
356 365 366
356 365 368
356 365 369
356 365 370
356 366 368
356 366 369
356 366 370
356 368 369
356 368 370
356 369 370
357 361 363
357 361 364
357 361 365
357 361 366
357 361 368
357 361 369
357 361 370
357 363 364
357 363 365
357 363 366
357 363 368
357 363 369
357 363 370
357 364 365
357 364 366
357 364 368
357 364 369
357 364 370
357 365 366
357 365 368
357 365 369
357 365 370
357 366 368
357 366 369
357 366 370
357 368 369
357 368 370
357 369 370
361 363 364
361 363 365
361 363 366
361 363 368
361 363 369
361 363 370
361 363 372
361 363 373
361 363 374
361 364 365
361 364 366
361 364 368
361 364 369
361 

1491 1494 1495
1491 1494 1496
1491 1494 1497
1491 1495 1496
1491 1495 1497
1491 1496 1497
1494 1495 1496
1494 1495 1497
1494 1496 1497
1495 1496 1497
1508 1509 1511
1508 1509 1512
1508 1509 1513
1508 1509 1514
1508 1511 1512
1508 1511 1513
1508 1511 1514
1508 1512 1513
1508 1512 1514
1508 1513 1514
1509 1511 1512
1509 1511 1513
1509 1511 1514
1509 1512 1513
1509 1512 1514
1509 1513 1514
1511 1512 1513
1511 1512 1514
1511 1513 1514
1512 1513 1514
length of train_set_MATRES: 4797
wsj_0032
102 105 106
105 106 108
108 110 111
length of train_set_MATRES: 4800
wsj_0918
1990 1991 1993
1990 1991 1997
1990 1993 1997
1991 1993 1997
1993 1997 1998
1993 1997 2000
1993 1998 2000
1997 1998 2000
1998 2000 2002
1998 2000 2005
1998 2002 2005
2000 2002 2005
2002 2005 2006
2017 2018 2020
2022 2023 2025
2022 2023 2028
2022 2025 2028
2023 2025 2028
2025 2028 2029
2025 2028 2030
2025 2029 2030
2028 2029 2030
2029 2030 2032
2032 2034 2035
2034 2035 2038
2038 2041 2042
2041 2042 2043
2041 2042 2044
2041 2043 

1194 1201 1203
1194 1202 1203
1201 1202 1203
1202 1203 1206
1206 1207 1208
1206 1207 1210
1206 1208 1210
1207 1208 1210
1207 1208 1214
1207 1210 1214
1208 1210 1214
1235 1238 1239
1235 1238 1240
1235 1239 1240
1238 1239 1240
1248 1250 1251
1250 1251 1253
1250 1251 1254
1250 1251 1256
1250 1253 1254
1250 1253 1256
1250 1254 1256
1251 1253 1254
1251 1253 1256
1251 1254 1256
1253 1254 1256
1274 1275 1278
1274 1275 1281
1274 1278 1281
1275 1278 1281
1278 1281 1282
1278 1281 1283
1278 1282 1283
1281 1282 1283
1310 1313 1315
1313 1315 1317
1313 1315 1318
1313 1317 1318
1315 1317 1318
1335 1336 1337
1335 1336 1340
1335 1336 1341
1335 1336 1344
1335 1337 1340
1335 1337 1341
1335 1337 1344
1335 1340 1341
1335 1340 1344
1335 1341 1344
1336 1337 1340
1336 1337 1341
1336 1337 1344
1336 1340 1341
1336 1340 1344
1336 1341 1344
1337 1340 1341
1337 1340 1344
1337 1341 1344
1340 1341 1344
1351 1358 1360
1358 1360 1361
1367 1370 1371
1370 1371 1373
1370 1371 1376
1370 1371 1378
1370 1373 1376
1370 1373 

In [32]:
MATRES_data_cases_train

[(415, 417, 418),
 (415, 417, 419),
 (415, 418, 419),
 (417, 418, 419),
 (419, 420, 421),
 (420, 421, 422),
 (420, 421, 423),
 (420, 422, 423),
 (421, 422, 423),
 (425, 426, 429),
 (426, 429, 431),
 (435, 437, 441),
 (437, 441, 442),
 (437, 441, 444),
 (437, 442, 444),
 (441, 442, 444),
 (187, 190, 191),
 (190, 191, 192),
 (190, 191, 194),
 (190, 191, 196),
 (190, 192, 194),
 (190, 192, 196),
 (190, 194, 196),
 (191, 192, 194),
 (191, 192, 196),
 (191, 194, 196),
 (192, 194, 196),
 (199, 200, 201),
 (200, 201, 202),
 (433, 436, 437),
 (438, 441, 445),
 (456, 458, 460),
 (460, 462, 463),
 (462, 463, 464),
 (462, 463, 465),
 (462, 464, 465),
 (463, 464, 465),
 (464, 465, 467),
 (482, 484, 485),
 (482, 484, 487),
 (482, 484, 488),
 (482, 485, 487),
 (482, 485, 488),
 (482, 487, 488),
 (484, 485, 487),
 (484, 485, 488),
 (484, 487, 488),
 (485, 487, 488),
 (2222, 2223, 2226),
 (2226, 2230, 2231),
 (2226, 2230, 2232),
 (2226, 2230, 2235),
 (2226, 2231, 2232),
 (2226, 2231, 2235),
 (2226, 22

In [20]:
len(eiid_pair_to_label_train)

255

In [19]:
for i in range(0, 10):
    if i < 5:
        print(i)
    else:
        continue
    if i < 5:
        print(i)

0
0
1
1
2
2
3
3
4
4


In [25]:
eiid_pair_to_label_train

{'ABC19980120.1830.0957': {(415, 417): 0,
  (415, 418): 0,
  (417, 418): 3,
  (415, 419): 0,
  (417, 419): 0,
  (418, 419): 0,
  (419, 420): 1,
  (419, 421): 1,
  (420, 421): 3,
  (420, 422): 0,
  (420, 423): 0,
  (421, 422): 0,
  (421, 423): 0,
  (422, 423): 0,
  (425, 426): 0,
  (425, 429): 1,
  (426, 429): 1,
  (426, 431): 1,
  (429, 431): 3,
  (431, 435): 3,
  (435, 437): 0,
  (435, 441): 0,
  (437, 441): 0,
  (437, 442): 1,
  (437, 444): 1,
  (441, 442): 1,
  (441, 444): 1,
  (442, 444): 3},
 'wsj_0791': {(179, 182): 1,
  (187, 190): 0,
  (187, 191): 1,
  (190, 191): 1,
  (190, 192): 1,
  (190, 194): 1,
  (190, 196): 0,
  (191, 192): 0,
  (191, 194): 0,
  (191, 196): 0,
  (192, 194): 3,
  (192, 196): 0,
  (194, 196): 0,
  (199, 200): 0,
  (199, 201): 3,
  (200, 201): 1,
  (200, 202): 0,
  (201, 202): 0},
 'VOA19980305.1800.2603': {(433, 436): 0,
  (433, 437): 0,
  (436, 437): 1,
  (438, 441): 0,
  (438, 445): 0,
  (441, 445): 2,
  (456, 458): 1,
  (456, 460): 1,
  (458, 460): 3,
 

In [44]:
my_dict = {1: "fight", 2: "help", 3: "jump"}
for i in my_dict.keys():
    for j in my_dict.keys():
        i = my_dict[i]
        j = my_dict[j]            

KeyError: 'fight'

In [39]:
%debug

> <ipython-input-38-9eecbac373a6>(4)<module>()
      2 for i in eiid_to_event_trigger_dict.keys():
      3     for j in eiid_to_event_trigger_dict.keys():
----> 4         i = eiid_to_event_trigger_dict[i]
      5         j = eiid_to_event_trigger_dict[j]
      6         print(i)

ipdb> i
'fight'
ipdb> q


In [49]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', unk_token='<unk>')
def RoBERTa_list(content, token_list = None, token_span_SENT = None):
    encoded = tokenizer.encode(content)
    roberta_subword_to_ID = encoded
    # input_ids = torch.tensor(encoded).unsqueeze(0)  # Batch size 1
    # outputs = model(input_ids)
    # last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
    roberta_subwords = []
    roberta_subwords_no_space = []
    for index, i in enumerate(encoded):
        r_token = tokenizer.decode([i])
        roberta_subwords.append(r_token)
        if r_token[0] == " ":
            roberta_subwords_no_space.append(r_token[1:])
        else:
            roberta_subwords_no_space.append(r_token)
    return roberta_subword_to_ID
    """
    roberta_subword_span = tokenized_to_origin_span(content, roberta_subwords_no_space[1:-1]) # w/o <s> and </s>
    roberta_subword_map = []
    if token_span_SENT is not None:
        roberta_subword_map.append(-1) # "<s>"
        for subword in roberta_subword_span:
            roberta_subword_map.append(token_id_lookup(token_span_SENT, subword[0], subword[1]))
        roberta_subword_map.append(-1) # "</s>" 
        return roberta_subword_to_ID, roberta_subwords, roberta_subword_span, roberta_subword_map
    else:
        return roberta_subword_to_ID, roberta_subwords, roberta_subword_span, -1
    """

In [51]:
con = "I love Florence."
RoBERTa_list(con)

[0, 38, 657, 9610, 4, 2]